In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip drive/My\ Drive/Datasets/SIH\ 2020/images.zip

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import sys
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Accessing Image Data

In [ ]:
training_data_dir = pathlib.Path('/content/newImg/training')
validation_data_dir = pathlib.Path('/content/newImg/cross_val/')
print(training_data_dir)
print(validation_data_dir)

In [ ]:
training_image_count = len(list(training_data_dir.glob('*/*.jpg')))
validation_image_count = len(list(validation_data_dir.glob('*/*.jpg')))
print('Total number of training images we have: ',training_image_count)
print('Total number of validation images we have: ',validation_image_count)

In [ ]:
CLASS_NAMES = np.array([item.name for item in training_data_dir.glob('*') if item.name != ".DS_Store" ])
CLASS_NAMES

## Loading Image Data

In [ ]:
train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                    rotation_range=45,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    horizontal_flip=True,
                    zoom_range=0.5)

In [ ]:
validation_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(training_image_count/BATCH_SIZE)

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(directory=str(training_data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(CLASS_NAMES))

In [ ]:
validation_data_gen = validation_image_generator.flow_from_directory(directory=str(validation_data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(CLASS_NAMES))

In [ ]:
#Testing
image,label = next(train_data_gen)
image.shape

In [ ]:
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')

In [ ]:
image_batch, label_batch = next(train_data_gen)
show_batch(image_batch, label_batch)

# Model

## Pretrained Model (Feature extractor)

In [ ]:
def getPretrainedModel(model='resnet50'):
    IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
    if model=='resnet50':
        base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                                    include_top=False,
                                                    weights='imagenet')
    elif model=='vgg16':
        base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                                    include_top=False,
                                                    weights='imagenet')
    elif model=='vgg19':
        base_model = tf.keras.applications.VGG19(input_shape = IMG_SHAPE,
                                               include_top = False,
                                               weights = 'imagenet')
    
    elif model=='mobileNet':
        base_model = tf.keras.applications.MobileNet(input_shape = IMG_SHAPE,
                                                     include_top = False,
                                                     weights = 'imagenet')    

    elif model=='inceptionV3':
        base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                                    include_top=False,
                                                    weights='imagenet')    
    elif model=='DenseNet121':
        base_model = tf.keras.applications.DenseNet121(input_shape=IMG_SHAPE,
                                                    include_top=False,
                                                    weights='imagenet')
    base_model.trainable = False
    return base_model

## Classifier

In [ ]:
def createModel():
    model = Sequential([
        Flatten(),
        Dense(1024, activation='relu'),   #added
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'), #added
        Dense(10, activation='softmax') 
    ])
    return model

#classifier_model = createModel()

## Select Base Model

In [ ]:
base_model = getPretrainedModel('mobileNet')
classifier_model = createModel()

In [ ]:
model = tf.keras.Sequential([
    base_model,
    classifier_model
])

optRms = tf.keras.optimizers.RMSprop(learning_rate=0.001)

optAdam = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optRms,
          loss='categorical_crossentropy',
          metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=training_image_count // BATCH_SIZE,
    epochs=20,
    validation_data = validation_data_gen,
    validation_steps=validation_image_count // BATCH_SIZE,
    shuffle=True
)

## Visualize training results

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')
"""
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
# plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()
"""


## Saving Model after every epoch

In [ ]:
saved_dir = "/content/drive/My Drive/"
name = 'mobileNet'
model.save_weights(saved_dir+name)
print(saved_dir+name)

## Load Model

In [ ]:
# Create a new model instance
model = createModel()

# Restore the weights
model.load_weights(saved_dir+name)

# Re-evaluate the model
loss,acc = model.evaluate(validation_data_gen)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## Export Trained Model

In [ ]:
tf.saved_model.save(model, "/content/exported_models")
!zip -r /content/exported_zip.zip /content/exported_models

In [ ]:
#Download exported ZIP file
from google.colab import files
files.download("/content/exported_zip.zip")

###Convert Model to Mobile model

In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)